# Data Loader

In [1]:
import codecs
from os import path
import pandas as pd
import pdb
import logging
import re
from gensim import parsing
import gensim
from gensim.parsing.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import classification_report

In [2]:
training_file = path.join('semeval2016-task6-trainingdata.txt')
training_data = pd.read_csv(training_file, header=0, delimiter='\t')

# Preprocessing

In [3]:
def preprocess(dataset):
    text=dataset["text"]
    #convert text to lower case
    text = text.lower()
   
    #removing whitespace
    text.strip()
   
    #removing digits
    text = gensim.parsing.preprocessing.strip_numeric(text)
    #text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    
    #remove stopwords
    text = gensim.parsing.preprocessing.remove_stopwords(text)
    
    #strip punctutation
    text = gensim.parsing.preprocessing.strip_punctuation(text)
    
    #strip multiple whitepsace that might occur after we remove stopwords
    text = gensim.parsing.preprocessing.strip_multiple_whitespaces(text)

    p = PorterStemmer()
    
    text = ' '.join(p.stem(word) for word in text.split())    

    #print(text)
    
    return text

In [ ]:
training_data

In [ ]:
train, test = train_test_split(training_data, test_size=0.3)

In [ ]:
train0, val = train_test_split(train, test_size=0.3)

In [ ]:
train0

# BERT

In [5]:
from transformers import AutoModel # For BERTs

model = AutoModel.from_pretrained("prajjwal1/bert-tiny") 

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") 

In [3]:
def tokenize_function(dataset):
    model_inputs = tokenizer(dataset['text'], padding="max_length", max_length=100, truncation=True)
    return model_inputs

# Abortion

In [14]:
from datasets import load_dataset

cache_dir = "./data_cache"

train_dataset = load_dataset(
    "tweet_eval",
    name="stance_abortion",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

val_dataset = load_dataset(
    "tweet_eval",
    name="stance_abortion",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

test_dataset = load_dataset(
    "tweet_eval",
    name="stance_abortion",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset)} instances loaded")

num_classes = np.unique(train_dataset['label']).size

Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Training dataset with 587 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Validation dataset with 66 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Test dataset with 280 instances loaded


In [15]:
# train_dataset=train_dataset.map(preprocess,batched=True)

In [16]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at ./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-ec684503b2709c7c.arrow
Loading cached processed dataset at ./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-29c6a235b83eca5d.arrow
Loading cached processed dataset at ./data_cache\tweet_eval\stance_abortion\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-9876f01e7dd29c6c.arrow


In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=num_classes)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at C:\Users\LYM/.cache\huggingface\transformers\3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.14.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_s

In [18]:
for param in model.bert.parameters():
    param.requires_grad = False

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="transformer_checkpoints",  # specify the directory where models weights will be saved a certain points during training (checkpoints)
    num_train_epochs=3,  # change this if it is taking too long on your computer
)  

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 587
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 222


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=222, training_loss=1.009326969181095, metrics={'train_runtime': 5.807, 'train_samples_per_second': 303.254, 'train_steps_per_second': 38.23, 'total_flos': 437114363400.0, 'train_loss': 1.009326969181095, 'epoch': 3.0})

In [21]:
import torch 
def predict_nn(trained_model, test_dataset):

    # Pass the required items from the dataset to the model    
    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))
        
    # the output dictionary contains logits, which are the unnormalised scores for each class for each example:
    pred_labs = np.argmax(output["logits"].detach().numpy(), axis=1)
    
    gold_labs = test_dataset["label"]
    
    return gold_labs, pred_labs

# Run the prediction function to get the results:
gold_labs, pred_labs = predict_nn(model, test_dataset)

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

acc = accuracy_score(gold_labs, pred_labs)
print(f'Accuracy = {acc}')

prec = precision_score(gold_labs, pred_labs, average='macro')
print(f'Precision (macro average) = {prec}')

rec = recall_score(gold_labs, pred_labs, average='macro')
print(f'Recall (macro average) = {rec}')

f1 = f1_score(gold_labs, pred_labs, average='macro')
print(f'F1 score (macro average) = {f1}')

print(classification_report(gold_labs, pred_labs))

Accuracy = 0.6678571428571428
Precision (macro average) = 0.30961070559610704
Recall (macro average) = 0.3352887048539222
F1 score (macro average) = 0.2811519078473722
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.68      0.98      0.80       189
           2       0.25      0.02      0.04        46

    accuracy                           0.67       280
   macro avg       0.31      0.34      0.28       280
weighted avg       0.50      0.67      0.55       280



In [19]:
test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 280
})

# Atheism

In [31]:
from datasets import load_dataset

cache_dir = "./data_cache"

train_dataset_atheism = load_dataset(
    "tweet_eval",
    name="stance_atheism",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

val_dataset_atheism = load_dataset(
    "tweet_eval",
    name="stance_atheism",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

test_dataset_atheism = load_dataset(
    "tweet_eval",
    name="stance_atheism",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset_atheism)} instances loaded")

num_classes = np.unique(train_dataset_atheism['label']).size

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

...

Dataset tweet_eval downloaded and prepared to ./data_cache\tweet_eval\stance_atheism\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.
Training dataset with 587 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_atheism\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Validation dataset with 66 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_atheism\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Test dataset with 220 instances loaded


In [33]:
type(train_dataset_atheism)

datasets.arrow_dataset.Dataset

In [34]:
train_dataset_atheism = train_dataset_atheism.map(tokenize_function, batched=True)
val_dataset_atheism = val_dataset_atheism.map(tokenize_function, batched=True)
test_dataset_atheism = test_dataset_atheism.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_atheism,
    eval_dataset=val_dataset_atheism,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 461
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 174


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=174, training_loss=0.9853380575947378, metrics={'train_runtime': 4.326, 'train_samples_per_second': 319.695, 'train_steps_per_second': 40.222, 'total_flos': 343287430200.0, 'train_loss': 0.9853380575947378, 'epoch': 3.0})

In [36]:
gold_labs, pred_labs = predict_nn(model, test_dataset_atheism)

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

acc = accuracy_score(gold_labs, pred_labs)
print(f'Accuracy = {acc}')

prec = precision_score(gold_labs, pred_labs, average='macro')
print(f'Precision (macro average) = {prec}')

rec = recall_score(gold_labs, pred_labs, average='macro')
print(f'Recall (macro average) = {rec}')

f1 = f1_score(gold_labs, pred_labs, average='macro')
print(f'F1 score (macro average) = {f1}')

print(classification_report(gold_labs, pred_labs))

Accuracy = 0.7272727272727273
Precision (macro average) = 0.24242424242424243
Recall (macro average) = 0.3333333333333333
F1 score (macro average) = 0.28070175438596495
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.73      1.00      0.84       160
           2       0.00      0.00      0.00        32

    accuracy                           0.73       220
   macro avg       0.24      0.33      0.28       220
weighted avg       0.53      0.73      0.61       220



D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

# Climate

In [38]:
from datasets import load_dataset

cache_dir = "./data_cache"

train_dataset_climate = load_dataset(
    "tweet_eval",
    name="stance_climate",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

val_dataset_climate = load_dataset(
    "tweet_eval",
    name="stance_climate",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

test_dataset_climate = load_dataset(
    "tweet_eval",
    name="stance_climate",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset_climate)} instances loaded")

num_classes = np.unique(train_dataset_climate['label']).size

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/133 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

...

Dataset tweet_eval downloaded and prepared to ./data_cache\tweet_eval\stance_climate\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.
Training dataset with 587 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_climate\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Validation dataset with 66 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_climate\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Test dataset with 169 instances loaded


In [39]:
train_dataset_climate = train_dataset_climate.map(tokenize_function, batched=True)
val_dataset_climate = val_dataset_climate.map(tokenize_function, batched=True)
test_dataset_climate = test_dataset_climate.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_climate,
    eval_dataset=val_dataset_climate,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 355
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 135


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=135, training_loss=1.2283945719401042, metrics={'train_runtime': 3.3796, 'train_samples_per_second': 315.123, 'train_steps_per_second': 39.945, 'total_flos': 264353661000.0, 'train_loss': 1.2283945719401042, 'epoch': 3.0})

In [41]:
gold_labs, pred_labs = predict_nn(model, test_dataset_climate)

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

acc = accuracy_score(gold_labs, pred_labs)
print(f'Accuracy = {acc}')

prec = precision_score(gold_labs, pred_labs, average='macro')
print(f'Precision (macro average) = {prec}')

rec = recall_score(gold_labs, pred_labs, average='macro')
print(f'Recall (macro average) = {rec}')

f1 = f1_score(gold_labs, pred_labs, average='macro')
print(f'F1 score (macro average) = {f1}')

print(classification_report(gold_labs, pred_labs))

Accuracy = 0.11242603550295859
Precision (macro average) = 0.34659659659659664
Recall (macro average) = 0.33423432935628056
F1 score (macro average) = 0.09651715874116518
              precision    recall  f1-score   support

           0       0.08      0.03      0.04        35
           1       0.07      0.91      0.13        11
           2       0.89      0.07      0.12       123

    accuracy                           0.11       169
   macro avg       0.35      0.33      0.10       169
weighted avg       0.67      0.11      0.11       169



# Feminist

In [43]:
from datasets import load_dataset

cache_dir = "./data_cache"

train_dataset_feminist = load_dataset(
    "tweet_eval",
    name="stance_feminist",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

val_dataset_feminist = load_dataset(
    "tweet_eval",
    name="stance_feminist",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

test_dataset_feminist = load_dataset(
    "tweet_eval",
    name="stance_feminist",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset_feminist)} instances loaded")

num_classes = np.unique(train_dataset_feminist['label']).size

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Dataset tweet_eval downloaded and prepared to ./data_cache\tweet_eval\stance_feminist\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.
Training dataset with 587 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_feminist\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Validation dataset with 66 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_feminist\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Test dataset with 285 instances loaded


In [44]:
train_dataset_feminist = train_dataset_feminist.map(tokenize_function, batched=True)
val_dataset_feminist = val_dataset_feminist.map(tokenize_function, batched=True)
test_dataset_feminist = test_dataset_feminist.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_feminist,
    eval_dataset=val_dataset_feminist,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 597
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 225


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=225, training_loss=1.0529041205512153, metrics={'train_runtime': 6.1137, 'train_samples_per_second': 292.949, 'train_steps_per_second': 36.803, 'total_flos': 444560945400.0, 'train_loss': 1.0529041205512153, 'epoch': 3.0})

In [46]:
gold_labs, pred_labs = predict_nn(model, test_dataset_feminist)

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

acc = accuracy_score(gold_labs, pred_labs)
print(f'Accuracy = {acc}')

prec = precision_score(gold_labs, pred_labs, average='macro')
print(f'Precision (macro average) = {prec}')

rec = recall_score(gold_labs, pred_labs, average='macro')
print(f'Recall (macro average) = {rec}')

f1 = f1_score(gold_labs, pred_labs, average='macro')
print(f'F1 score (macro average) = {f1}')

print(classification_report(gold_labs, pred_labs))

Accuracy = 0.6421052631578947
Precision (macro average) = 0.21403508771929824
Recall (macro average) = 0.3333333333333333
F1 score (macro average) = 0.2606837606837607
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.64      1.00      0.78       183
           2       0.00      0.00      0.00        58

    accuracy                           0.64       285
   macro avg       0.21      0.33      0.26       285
weighted avg       0.41      0.64      0.50       285



D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

# Hillary

In [48]:
from datasets import load_dataset

cache_dir = "./data_cache"

train_dataset_hillary = load_dataset(
    "tweet_eval",
    name="stance_hillary",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

val_dataset_hillary = load_dataset(
    "tweet_eval",
    name="stance_hillary",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Validation dataset with {len(val_dataset)} instances loaded")

test_dataset_hillary = load_dataset(
    "tweet_eval",
    name="stance_hillary",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset_hillary)} instances loaded")

num_classes = np.unique(train_dataset_hillary['label']).size

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/25.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/233 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Dataset tweet_eval downloaded and prepared to ./data_cache\tweet_eval\stance_hillary\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.
Training dataset with 587 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_hillary\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Validation dataset with 66 instances loaded


Reusing dataset tweet_eval (./data_cache\tweet_eval\stance_hillary\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


Test dataset with 295 instances loaded


In [49]:
train_dataset_hillary = train_dataset_hillary.map(tokenize_function, batched=True)
val_dataset_hillary = val_dataset_hillary.map(tokenize_function, batched=True)
test_dataset_hillary = test_dataset_hillary.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hillary,
    eval_dataset=val_dataset_hillary,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 620
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 234


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=234, training_loss=0.9976687961154513, metrics={'train_runtime': 5.4765, 'train_samples_per_second': 339.63, 'train_steps_per_second': 42.728, 'total_flos': 461688084000.0, 'train_loss': 0.9976687961154513, 'epoch': 3.0})

In [51]:
gold_labs, pred_labs = predict_nn(model, test_dataset_hillary)

In [52]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

acc = accuracy_score(gold_labs, pred_labs)
print(f'Accuracy = {acc}')

prec = precision_score(gold_labs, pred_labs, average='macro')
print(f'Precision (macro average) = {prec}')

rec = recall_score(gold_labs, pred_labs, average='macro')
print(f'Recall (macro average) = {rec}')

f1 = f1_score(gold_labs, pred_labs, average='macro')
print(f'F1 score (macro average) = {f1}')

print(classification_report(gold_labs, pred_labs))

Accuracy = 0.5830508474576271
Precision (macro average) = 0.1943502824858757
Recall (macro average) = 0.3333333333333333
F1 score (macro average) = 0.24553890078515347
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        78
           1       0.58      1.00      0.74       172
           2       0.00      0.00      0.00        45

    accuracy                           0.58       295
   macro avg       0.19      0.33      0.25       295
weighted avg       0.34      0.58      0.43       295



D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\LYM\miniconda3\envs\data_analytics_course0\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 